In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dataset = pd.read_excel(r'/content/drive/MyDrive/Relatas/sales-pipeline.xlsx')

Reading the first dataset

In [ ]:
dataset.head()

,Account,Opportunity_ID,Sales_Agent,SalesAgentEmailID,ContactEmailID,Deal_Stage,Product,Close_Value,Created Date,Close Date,DateDiff
0,Konex,3KR50X38,Corliss Cosme,corliss@piedpiper.com,delila@konex.com,Won,GTX Plus Basic,1255,2016-01-04,2016-05-24,141
1,Finjob,YFS5KLJJ,Rosalina Dieter,rosalina@piedpiper.com,belinda@finjob.com,Won,MG Special,45,2016-01-04,2016-11-02,303
2,Kinnamplus,N0ONCYVZ,Donn Cantrell,donn@piedpiper.com,monte@kinnamplus.com,Lost,MG Special,1054,2016-01-06,2016-12-07,336
3,Genco Pura Olive Oil Company,IBEUTCSW,James Ascencio,james@piedpiper.com,karole@gencopuraoliveoilcompany.com,In Progress,MG Advanced,1054,2016-01-06,2016-09-11,249
4,Initech,G8AO5FA2,Wilburn Farren,wilburn@piedpiper.com,alejandro@initech.com,Lost,MG Advanced,517,2016-01-06,2016-09-18,256


Checking for any empty values

In [ ]:
dataset.isnull().sum()

Account              0
Opportunity_ID       0
Sales_Agent          0
SalesAgentEmailID    0
ContactEmailID       0
Deal_Stage           0
Product              0
Close_Value          0
Created Date         0
Close Date           0
DateDiff             0
dtype: int64

Reading the second dataset

In [ ]:
dataset_extra = pd.read_excel(r'/content/drive/MyDrive/Relatas/interactions.xlsx')

In [ ]:
dataset_extra.head()

,interactionID,fromEmailId,toEmailId,InteractionDate,InteractionType(Call/Email/SMS),Extracted Interaction Text
0,8842411,gladys@piedpiper.com,kristle@dontechi.com,2016-01-04,Call,I thoroughly enjoyed your presentation yesterd...
1,8015509,donn@piedpiper.com,orpha@umbrellacorporation.com,2016-01-04,Call,speaking with you and your staff will be a tre...
2,5976842,kary@piedpiper.com,claribel@goodsilron.com,2016-01-04,Email,"However, I am unable to join because of my fam..."
3,1471794,boris@piedpiper.com,gordon@scotfind.com,2016-01-04,Call,We are extremely sorry to inform you that we h...
4,4330320,darcel@piedpiper.com,benny@treequote.com,2016-01-04,Call,I don't think it's possible for me to meet you


In [ ]:
dataset_extra.isnull().sum()

interactionID                      0
fromEmailId                        0
toEmailId                          0
InteractionDate                    0
InteractionType(Call/Email/SMS)    0
Extracted Interaction Text         0
dtype: int64

Selecting relevant columns for the dataframe to be used

In [ ]:
df_extra = dataset_extra[['InteractionDate','Extracted Interaction Text']]

In [ ]:
df_extra.head()

,InteractionDate,Extracted Interaction Text
0,2016-01-04,I thoroughly enjoyed your presentation yesterd...
1,2016-01-04,speaking with you and your staff will be a tre...
2,2016-01-04,"However, I am unable to join because of my fam..."
3,2016-01-04,We are extremely sorry to inform you that we h...
4,2016-01-04,I don't think it's possible for me to meet you


Using nltk in order to import Sentiment Intensity Analyser as we'll be using that NLP algorithm in order to convert the Extracted interaction text to a factor influencing the revenue.

In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
analyser = SentimentIntensityAnalyzer()
df_extra.reset_index(inplace=True, drop=True)
df_extra[['neg', 'neu', 'pos', 'compound']] = df_extra['Extracted Interaction Text'].apply(analyser.polarity_scores).apply(pd.Series)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:
df_extra.head(20)

,InteractionDate,Extracted Interaction Text,neg,neu,pos,compound
0,2016-01-04,I thoroughly enjoyed your presentation yesterd...,0.039,0.722,0.238,0.9176
1,2016-01-04,speaking with you and your staff will be a tre...,0.000,0.837,0.163,0.8126
2,2016-01-04,"However, I am unable to join because of my fam...",0.071,0.756,0.173,0.4287
3,2016-01-04,We are extremely sorry to inform you that we h...,0.069,0.819,0.112,0.2516
4,2016-01-04,I don't think it's possible for me to meet you,0.000,1.000,0.000,0.0000
5,2016-01-04,I have'nt heard from you regarding the status ...,0.172,0.828,0.000,-0.6124
6,2016-01-04,I am very pleased to accept your invitation to...,0.000,0.758,0.242,0.7233
7,2016-01-04,Congratulations! Your work group achieved the ...,0.000,0.789,0.211,0.9564
8,2016-01-04,I am sorry that I will be unable to schedule a...,0.064,0.936,0.000,-0.0772
9,2016-01-04,Unfortunately we cannot approve the proposal t...,0.179,0.821,0.000,-0.4019


In [ ]:
df_extra = df_extra[df_extra['compound'] > 0.5]

In [ ]:
df_extra.head()

,InteractionDate,Extracted Interaction Text,neg,neu,pos,compound
0,2016-01-04,I thoroughly enjoyed your presentation yesterd...,0.039,0.722,0.238,0.9176
1,2016-01-04,speaking with you and your staff will be a tre...,0.000,0.837,0.163,0.8126
6,2016-01-04,I am very pleased to accept your invitation to...,0.000,0.758,0.242,0.7233
7,2016-01-04,Congratulations! Your work group achieved the ...,0.000,0.789,0.211,0.9564
11,2016-01-04,see how cutting edge technology can empower yo...,0.043,0.652,0.306,0.8516


In [ ]:
df_pred = df_extra[df_extra['InteractionDate'] > '2018-12-31']

In [ ]:
df_pred.head()

,InteractionDate,Extracted Interaction Text,neg,neu,pos,compound
264829,2019-01-01,We have read your innovative plans to improve ...,0.000,0.603,0.397,0.9730
264830,2019-01-01,Please accept my regrets in resigning from my ...,0.173,0.512,0.315,0.5574
264832,2019-01-01,I appreciate your contribution to Morgan Stanl...,0.000,0.673,0.327,0.9118
264833,2019-01-01,"As the manager of our computer department, I c...",0.000,0.763,0.237,0.7351
264834,2019-01-01,"On behalf of management, I congratulate you on...",0.000,0.617,0.383,0.9808


In [ ]:
df_train = df_extra[df_extra['InteractionDate'] < '2019-01-01']

In [ ]:
df_pred = df_pred[['InteractionDate','compound']]
df_train = df_train[['InteractionDate', 'compound']]

In [ ]:
dataset.head()

,Account,Opportunity_ID,Sales_Agent,SalesAgentEmailID,ContactEmailID,Deal_Stage,Product,Close_Value,Created Date,Close Date,DateDiff
0,Konex,3KR50X38,Corliss Cosme,corliss@piedpiper.com,delila@konex.com,Won,GTX Plus Basic,1255,2016-01-04,2016-05-24,141
1,Finjob,YFS5KLJJ,Rosalina Dieter,rosalina@piedpiper.com,belinda@finjob.com,Won,MG Special,45,2016-01-04,2016-11-02,303
2,Kinnamplus,N0ONCYVZ,Donn Cantrell,donn@piedpiper.com,monte@kinnamplus.com,Lost,MG Special,1054,2016-01-06,2016-12-07,336
3,Genco Pura Olive Oil Company,IBEUTCSW,James Ascencio,james@piedpiper.com,karole@gencopuraoliveoilcompany.com,In Progress,MG Advanced,1054,2016-01-06,2016-09-11,249
4,Initech,G8AO5FA2,Wilburn Farren,wilburn@piedpiper.com,alejandro@initech.com,Lost,MG Advanced,517,2016-01-06,2016-09-18,256


In [ ]:
df1 = dataset[['Deal_Stage', 'Close_Value', 'Created Date', 'DateDiff']]

In [ ]:
df1 = df1[df1['Deal_Stage'] == 'Won']

In [ ]:
df1.head()

,Close_Value,InteractionDate,DateDiff
0,1255,2016-01-04,141
1,45,2016-01-04,303
5,3180,2016-01-07,24
6,517,2016-01-08,335
7,5539,2016-01-10,328


In [ ]:
df1 = df1[['Close_Value','Created Date', 'DateDiff']]

In [ ]:
df1['compound'] = 0

In [ ]:
df_train['Count'] = 1

In [ ]:
df_train['InteractionDate'] = pd.to_datetime(df_train['InteractionDate'])

df_new = df_train.groupby(df_train['InteractionDate'].dt.date).sum()

In [ ]:
df_new.head()

,compound
InteractionDate,
2016-01-04,0.833273
2016-01-05,0.822345
2016-01-06,0.846976
2016-01-07,0.817393
2016-01-08,0.838793


In [ ]:
df1.rename(columns={'Created Date':'InteractionDate'}, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
df_new['compound'] = df_new['compound']/df_new['Count']

Couldn't figure out how to merge the 2 dataframes without getting rid of duplicates in 'Created date', so grouping all of it together which makes DateDiff useless for the time being.

In [ ]:
df1['InteractionDate'] = pd.to_datetime(df1['InteractionDate'])

df_grouped_df1 = df1.groupby(df1['InteractionDate'].dt.date).sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_new.head()

,compound
InteractionDate,
2016-01-04,0.833273
2016-01-05,0.822345
2016-01-06,0.846976
2016-01-07,0.817393
2016-01-08,0.838793


In [ ]:
merge = pd.merge(df_new, df_grouped_df1, how = 'outer', on = 'InteractionDate')

In [ ]:
merge = merge.replace(np.nan, 0.0)

In [ ]:
merge.head()

,InteractionDate,compound,Close_Value,DateDiff
InteractionDate,,,,
2016-01-04,0.0,0.833273,1300.0,444.0
2016-01-05,0.0,0.822345,0.0,0.0
2016-01-06,0.0,0.846976,0.0,0.0
2016-01-07,0.0,0.817393,3180.0,24.0
2016-01-08,0.0,0.838793,517.0,335.0


In [ ]:
merge = merge.reindex(columns=['InteractionDate','compound','Close_Value','DateDiff'])

In [ ]:
from sklearn import linear_model

In [ ]:
X_train = merge[['compound','InteractionDate']]
y_train = merge['Close_Value']

In [ ]:
regress = linear_model.LinearRegression()
regress.fit(X_train, y_train)

LinearRegression()

In [ ]:
df_pred.head()

,InteractionDate,compound
264829,2019-01-01,0.9730
264830,2019-01-01,0.5574
264832,2019-01-01,0.9118
264833,2019-01-01,0.7351
264834,2019-01-01,0.9808


In [ ]:
import datetime as dt
merge['InteractionDate'] = pd.to_datetime(merge['InteractionDate'])
merge['InteractionDate']=merge['InteractionDate'].map(dt.datetime.toordinal)

In [ ]:
df_predict['InteractionDate'] = pd.to_datetime(df_predict['InteractionDate'])
df_predict['InteractionDate']=df_predict['InteractionDate'].map(dt.datetime.toordinal)

In [ ]:
df_pred.head()

,InteractionDate,compound
264829,2019-01-01,0.9730
264830,2019-01-01,0.5574
264832,2019-01-01,0.9118
264833,2019-01-01,0.7351
264834,2019-01-01,0.9808


In [ ]:
df_pred['InteractionDate'] = pd.to_datetime(df_pred['InteractionDate'])

df_predict = df_pred.groupby(df_pred['InteractionDate'].dt.date).sum()

In [ ]:
df_predict.head()

,InteractionDate,compound
0,737060,0.828846
1,737061,0.817134
2,737062,0.829175
3,737063,0.817763
4,737064,0.820829


In [ ]:
df_predict['compound'] = df_predict['compound']/df_predict['Count']

In [ ]:
df_predict = df_predict.reset_index()

In [ ]:
df_predict['InteractionDate'] = pd.to_datetime(df_predict['InteractionDate'])
df_predict['InteractionDate']=df_predict['InteractionDate'].map(dt.datetime.toordinal)

In [ ]:
Prediction = regress.predict([[]])

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  "X does not have valid feature names, but"


ValueError: ignored